In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client, LocalCluster

client = Client() # Note that `memory_limit` is the limit **per worker**.
# n_workers=4,
#                 threads_per_worker=1,
#                 memory_limit='3GB'
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [ ]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv
import scipy.constants

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

In [ ]:
# file path to data and configs

prefixes = []

#prefix = "/home/radioglaciology/thomas/radar_data/20230721-summit-day1-bench/20230721_104552"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103641"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103845" # 6 db higher tx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104059"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104248" # another 6 db
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104456" # 3db higher rx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104742" # 20 -> 30 mhz chirp bw
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105025" # bw back to 20, 15 mhz lo offset
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105509") # 100k pulses, 30 us transmit, 90 us rx
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105921") # 100k pulses, 10 us transmit, 90 us rx
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_110253") # 100k pulses, 5 us transmit, 60 us rx

prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_145418")
prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_150425")
prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_150853")

# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_151432") # 20 us transmit
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_153254") # 10 us transmit
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_152404") #  5 us transmit

raw_datasets = []
for prefix in prefixes:
    # resave data as zarr for dask processing
    zarr_path = pr.save_radar_data_to_zarr(prefix)

    # open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
    raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})
    raw_datasets.append(raw)

In [ ]:
zero_sample_idx = 36 # X310, fs = 20 MHz
#zero_sample_idx = 63 # X310, fs = 50 MHz
#zero_sample_idx = 159 # B205mini, fs = 56 MHz

modify_rx_window = True # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "blackman" # what you want to change the rx window to if modify_rx_window is true

#dielectric_constant = 2.2957
dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

n_stack = 60000 #900000

In [ ]:
plots = []

for raw in raw_datasets:
    # Generate reference chirp
    if modify_rx_window:
        config = copy.deepcopy(raw.config)
        config['GENERATE']['window'] = rx_window
    else:
        config = raw.config

    chirp_ts, ref_chirp = generate_chirp(config)

    # Stack and pulse compress
    stacked = pr.stack(raw, n_stack)
    compressed = pr.pulse_compress(stacked, ref_chirp,
                                fs=stacked.config['GENERATE']['sample_rate'],
                                zero_sample_idx=zero_sample_idx,
                                signal_speed=sig_speed)

    compressed_power = xr.apply_ufunc(
        lambda x: 20*np.log10(np.abs(x)),
        compressed,
        dask="parallelized"
    )

    plots.append(compressed_power.radar_data[0,:].hvplot.line(label=f"{os.path.basename(raw.prefix)}"))

In [ ]:
plot1D = hv.Overlay(plots)
plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', height=600, title=f"1D Radargram Multi-File Comparison")
plot1D = plot1D.opts(xlim=(0, 500), ylim=(-80, 0), show_grid=True)
plot1D

In [ ]:
#hvplot.save(plot1D, f"outputs/tmp.png")